In [7]:
import pandas as pd
from utils import data_wrangling as dw
from models import autoarima_model as am
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

In [9]:
INIT = "2023-01-01"
END = "2023-02-28"

In [10]:
load = dw.ons_data(freq='h', ano_inicio=2012, ano_fim=2023, idreg="S")
df_load = dw.pipeline(load).loc[INIT:END,:]
df_load.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1416 entries, 2023-01-01 00:00:00 to 2023-02-28 23:00:00
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   id_reg      1416 non-null   category
 1   desc_reg    1416 non-null   category
 2   load_mwmed  1416 non-null   float64 
dtypes: category(2), float64(1)
memory usage: 25.3 KB


In [11]:
df_load.describe()

,load_mwmed
count,1416.000000
mean,13059.200058
std,2355.848939
min,6680.465000
25%,11374.152500
50%,13195.500000
75%,14796.673500
max,18345.403000


In [12]:
am.auto_arima_model(df=df_load, h_=24)

,ds,AutoARIMA
unique_id,,
hourly_load,2023-03-01 00:00:00,12577.446289
hourly_load,2023-03-01 01:00:00,11857.718750
hourly_load,2023-03-01 02:00:00,11368.575195
hourly_load,2023-03-01 03:00:00,11105.305664
hourly_load,2023-03-01 04:00:00,11055.261719
hourly_load,2023-03-01 05:00:00,11537.592773
hourly_load,2023-03-01 06:00:00,12288.075195
hourly_load,2023-03-01 07:00:00,12992.813477
hourly_load,2023-03-01 08:00:00,13389.875977


In [32]:
df_load_sf.head()

,ds,y,unique_id
0,2012-01-01 00:00:00,7253.940,hourly_load
1,2012-01-01 01:00:00,6890.371,hourly_load
2,2012-01-01 02:00:00,6414.702,hourly_load
3,2012-01-01 03:00:00,6093.842,hourly_load
4,2012-01-01 04:00:00,5908.571,hourly_load


In [30]:
df_load_sf.dtypes

ds           datetime64[ns]
y                   float64
unique_id            object
dtype: object

In [33]:
sf = StatsForecast(
    models=[AutoARIMA(season_length=24)],
    freq='H'
)

In [34]:
sf.fit(df_load_sf)